In [1]:
## import pandas as pd
import numpy as np
# from tqdm.autonotebook import tqdm
import re, nltk, random, os, json
import pandas as pd
import http.client, urllib.request, urllib.parse, urllib.error, base64
from nltk.corpus import stopwords
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from rank_bm25 import BM25Okapi
import time, os, pickle
# keras
from keras.layers import Input, LSTM, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate, Embedding, Multiply, Dot, Dense, Subtract, Activation, SimpleRNN, Flatten, Lambda
from keras.models import Model
from keras import backend as K
import numpy as np


/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/sklearn/feature_extraction/dict_vectorizer.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
Using TensorFlow backend.


In [2]:
def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }
    
    queries = [str(i).replace("\'", "") for i in queries]

    params = urllib.parse.urlencode({})
    
    while True:
        try:
            conn = http.client.HTTPSConnection('api.msturing.org')
    #         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
            conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)
            conn.close()
            break
        except Exception as e:
    #         print(data)
            time.sleep(5)
#         print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

In [305]:
queries = [i if i[-1] != " " else i[:-1] for i in pd.read_csv("data/cedr/query.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()]
properties = pd.read_csv("data/cedr/doc.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
wikis = pd.read_csv("data/cedr/wikipedia1.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
questions = pd.read_csv("data/cedr/question-qq1.tsv", sep="\t", names=["q", "qid", "text"]).text.unique().tolist()
data = getTermMSvec(queries + properties + wikis + questions)

In [306]:
saveDict(data, "data/cedr/query-doc-wiki-question1.pkg")

In [44]:
stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.,]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc,enableStopword=False):
    doc = regex.sub(' ', doc)
    if enableStopword:
        doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower()

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    result = {}
    
    
    
    for i in [5, 10 ,15, 20]:
        metric = MSnDCG(xrelnum, grades, cutoff=i)
        result["ndcg@%d" % i] = metric.compute(labeled_ranked_list)
        
        nerr = nERR(xrelnum, grades, cutoff=i)
        result["nerr@%d" % i] = nerr.compute(labeled_ranked_list)
        
        
        _ranked_list = ranked_list[:i]
        result["p@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(_ranked_list)
        result["r@%d" % i] = len(set.intersection(set(qrels.keys()), set(_ranked_list))) / len(qrels)
    result["rp"] = len(set.intersection(set(qrels.keys()), set(ranked_list[:len(qrels)]))) / len(qrels)
    
    return result



def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0) for m in matches]).lower()

def getTermMSvec(all_properties):
    tmp = {}
    for i in range(0, len(all_properties), 20):
        data = getVectors(all_properties[i:i+20])
        for i in data:
            tmp[i] = data[i]
    return tmp

def saveDict(d, name):
    f = open(name,"wb")
    pickle.dump(d,f)
    f.close()
def loadDict(name):
    return pickle.load( open(name, "rb" ) )

In [19]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")
df_action = pd.read_csv("data/AKG/Test Collection/AM/akg_standard_am_verb_object_rele.csv")

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})
for c in ["query", "entityType", "action", "entity"]:
    topic[c] = topic[c].str.lower().replace("\'", "")
    
df = df.merge(topic, how="inner", on="query_id")
# df['query'] = df[['action', 'entity', 'entityType']].astype(str).apply(' '.join, axis=1)

with open("data/AKG/Formal Run Topics/AM_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, entity, url = [], [], []
    for p in data['queries'][0]:
        qid.append(p['queryId'])
        entity.append(p['entity'].lower())
        url.append(p['entityurl'].split("/")[-1])
topic2 = pd.DataFrame({"url":url, "entity": entity})
df = df.merge(topic2, how="inner", on="entity")

In [5]:
trainIds, testIds = [], []
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() > 1:
        ids = list(group.query_id.unique())
        mid = int(group.query_id.nunique() / 2)
        trainIds.extend(ids[:mid])
        testIds.extend(ids[mid:])
    else:
        ids = list(group.query_id.unique())
        trainIds.extend(ids)

In [205]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to ", "")
for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
    df_wiki[col] = [preprocessingText(str(i), True) for i in df_wiki[col]]
    
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [13]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value
with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']

qids = []
props = []
for qid in run['1']:
    tmp = list(run['1'][str(qid)].keys())
    qids.extend([int(qid)] * len(tmp))
    props.extend(tmp)
df_run = pd.DataFrame({"query_id": qids, "property": props})
df_run = df_run.merge(topic, how="left", on="query_id")

In [14]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)

In [78]:
dfp = df[["query_id", "entityType", "property"]].append(df_run[["query_id", "entityType", "property"]])
_dfp = dfp[dfp.query_id.isin(trainIds)]
type2prop = _dfp.groupby("entityType")['property'].unique().to_dict()
prop2popularity = _dfp.groupby("property").size().to_dict()
type2prop2popularity = _dfp.groupby(["entityType", "property"]).size().to_dict()

In [ ]:
terms = []
for c in [ 'entity', 'entityType', 'action']:
    for i in df[c].unique().tolist():
        terms.append(i.replace("\'", ""))
terms.extend([camel_case_split(i) for i in df.property.unique()])
term2MSvec = getTermMSvec(terms)

In [17]:
dfw = df_wiki[df_wiki.title.str.contains("|".join(df.entity.unique().tolist()))]

In [15]:
entityInWiki = []
for e in df.entity.unique():
    tmp = dfw[dfw.title.str.contains(e)]
    if len(tmp) > 0:
        entityInWiki.append(e)

NameError: name 'dfw' is not defined

In [ ]:
qidInWiki = []
for e in entityInWiki:
    qidInWiki.append(df[df.entity == e].query_id.tolist()[0])
saveDict(qidInWiki, "qidInWiki")

In [225]:
corpus = []
doc2title = {}
for name, group in df_wiki.groupby("title"):
    doc = []
    for col in ["title", "overview", "sectionLabel", "headline", "text"]:
        doc.append(" ".join(group[col].unique().tolist()))
        
    doc = "\t".join(doc)
    corpus.append(doc)
    doc2title[doc] = name
    
bm25 = BM25Okapi([tokenizer.tokenize(doc) for doc in corpus])

In [286]:
corpus[0]

'180 scooter\tdoing 180 means rotating scooter 180 degrees. this basic fundamental trick scooter. it tends one first tricks learned basic ollie jump essential learning tricks. with practice expect add trick repertoire move scooter tricks time.\tlanding 180 landing 180 barrier performing faux 180\tscout locations. practice safety. gain momentum. compress knees. do ollie. initiate 180. stick landing. find barrier mid shin level. practice ollieing barrier. perform ollie. land 180. understand faux 180. practice stationary. hop spin. land trick.\tfind open area free hazards uneven surfaces sharp objects pedestrians . skate parks great places practice tricks. also large open parking lots used tricks. make sure rules skating scooters if shy stick driveway garage till feel comfortable. some might poke fun wear knee elbow pads joints worth always wear helmet. you prone injury first practice complex tricks like 180. don increase chances needing knee surgery push quick bursts reach fast speed. th

In [74]:
# load : qid2MSvec.pkl
import pickle

qid2MSvec = pickle.load( open( "qid2MSvec.pkl", "rb" ) )
# qid2MSvec = {}
# for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
#     query = row['action'] + " " + row['entity']
#     if row['query_id'] in qid2MSvec:
#         continue
#     if row['entity'] not in entityInWiki:
#         data = getVectors([query])
#         qid2MSvec[row['query_id']] = list(data.values())
#         continue        
# #     titles = [str(i).replace("\'", "") for i in dfw[dfw.title.str.contains(row['entity'])].text.unique().tolist()]
#     titles = dfw[dfw.title.str.contains(row['entity'])].title.unique().tolist()
#     data = getTermMSvec([query] + titles)
#     rank = {}
#     for i in data:
#         if i == query:
#             continue
#         rank[i] = cosine_similarity([data[query]], [data[i]])[0][0]
#     best_title = sorted(rank.items(), key=lambda x: x[1])[-1][0]
#     sentences = []
#     tokens = " ".join(dfw[dfw.title == best_title].text.tolist()).split()
#     for i in range(0, len(tokens), 10):
#         sentences.append(" ".join(tokens[i:i+10]))
#     data = getTermMSvec([query] + sentences)
#     qid2MSvec[row['query_id']] = list(data.values())


In [296]:
# Get questions
import glob
from run_scraper import crawl
# from tqdm.autonotebook import tqdm

# finish = []
# for file in glob.glob("akg/*"):
#     finish.append(file.split("/")[1].replace("_", " ").replace(".csv", ""))
bestTitle2qid = loadDict("bestTitle2qid.pkl")
# bestTitle2qid = {}
# for idx, row in tqdm(df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows(), total=229):
#     query = row['action'] + " " + row['entity']
#     if row['entity'] not in entityInWiki:
#         continue        
#     titles = dfw[dfw.title.str.contains(row['entity'])].title.unique().tolist()
#     data = getTermMSvec([query] + titles)
#     rank = {}
#     for i in data:
#         if i == query:
#             continue
#         rank[i] = cosine_similarity([data[query]], [data[i]])[0][0]
#     best_title = sorted(rank.items(), key=lambda x: x[1])[-1][0]
#     bestTitle2qid[best_title] = row['query_id']
#     try:
#         crawl(best_title, "akg/", "no_question")
#     except:
#         print(best_title)
            


In [250]:
questions

['what mean benchmarking what benchmarking used what four types benchmarking',
 'which supplements i take bodybuilding does creatine cause hair loss when i take bodybuilding supplements',
 'what 4 rules brainstorming what another word brainstorming how brainstorm',
 'what considered chain smoking is ok smoke 2 cigarettes day how many cigarettes chain smoker smoke day',
 'how get funding charity how i apply charitable grant what source funding',
 'can i dye hair brown bleaching does dying hair brown damage how dye blonde hair back brown',
 'stop scene accident hit run',
 'why difficult learn foreign language is english hard foreigners learn what best age learning foreign language',
 'how i successful mediation what success rate mediation what 5 steps mediation',
 'how aerobic exercise affect muscles is running aerobic anaerobic what decreases aerobic exercise',
 'what spoofing attack what mean spoofing how spoofing attack occur',
 'how spray paint fabric can put fabric paint spray bottl

In [77]:
import glob
# qid2questions2MSvec = {}
# for file in glob.glob("akg/*"):
#     title = file.split("/")[1].replace("_", " ").replace(".csv", "")
#     lines = open(file).read().splitlines()
#     questions = []
#     for i in lines[1:]:
#         questions.extend(i.split(";"))
#     title2questions[title] = list(set(questions))
#     data = getVectors([title] + list(set(questions)) )
#     qid2questions2MSvec[bestTitle2qid[title]] = list(data.values())
qid2questions2MSvec = loadDict("qid2questions2MSvec")

In [78]:
# qid2query2MSvec = {}
# queries = []
# for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
#     query = preprocessingText(row['action'] + " " + row['entity'] + " " + row['entityType'])
#     qid2query2MSvec[row['query_id']] = getVectors([query])[query]
qid2query2MSvec = pickle.load( open( "qid2query2MSvec.pkl", "rb" ) )

In [79]:
# properties = []
# for i in run["1"]:
#     properties.extend(run["1"][i].keys())
# properties = list(set(properties + df.property.unique().tolist()))
# prop2MSvec = getTermMSvec(properties)
prop2MSvec = pickle.load( open( "prop2MSvec.pkl", "rb" ) )

In [81]:
# CLF with Question MS vectors, Two DSSM
MAX_QUESTION_LENGHT = np.max([len(qid2questions2MSvec[i]) for i in qid2questions2MSvec])
MAX_SEQUENCE_LENGHT = int(np.max([len(qid2MSvec[i]) for i in qid2MSvec])) + 1
# def runDocEval(qid2MSvec, prop2MSvec):
class CLF():
    def __init__(self):

        self.queryInput = Input(shape=(MAX_SEQUENCE_LENGHT, 100,))
        self.questionInput = Input(shape=(MAX_QUESTION_LENGHT, 100,))
        
        self.propPosInput = Input(shape=(100,))
        lstm = LSTM(100)

        queryEmbeddingLayer = Dense(100, name="uEmb")
        questionEmbeddingLayer = Dense(100, name="qEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

#         qEmb = queryEmbeddingLayer(lstm(self.queryInput))
        qEmb = queryEmbeddingLayer(self.queryInput)
        qtEmb = questionEmbeddingLayer(self.questionInput)
        pEmb = propEmbeddingLayer(self.propPosInput)
        
        qEmb = Concatenate(axis=1)([qEmb, qtEmb])

        dense = Dense(1, activation="sigmoid")

        pred = Multiply()([qEmb, pEmb])
        
        self.pred = dense(GlobalMaxPooling1D()(pred))
#         self.pred = dense(pred)

        self.model = Model(inputs=[self.queryInput, self.questionInput, self.propPosInput], outputs=self.pred)
        self.model.compile(optimizer="adam", loss="binary_crossentropy")

    def generate_train_data(self, df):
        x_query, x_question, x_pos_prop, y = [], [], [], []
        for name, group in df.groupby("query_id"):
            cand_pos_prop = group.property.tolist()
            rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()
            for idx, row in group.iterrows():
                if name in qid2questions2MSvec:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
                    x_question.append(qid2questions2MSvec[name])
                else:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
                    x_question.append([])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[row['property']])
                y.append(1)
        
            cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
#             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
            for neg_prop in cand_neg_prop:
                if name in qid2questions2MSvec:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
                    x_question.append(qid2questions2MSvec[name])
                    
                else:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
                    x_question.append([])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[neg_prop])
                y.append(0)

        x_query = pad_sequences(x_query,dtype='float32', maxlen=MAX_SEQUENCE_LENGHT)
        x_question = pad_sequences(x_question,dtype='float32', maxlen=MAX_QUESTION_LENGHT)
        x_pos_prop = np.array(x_pos_prop)
        return [x_query, x_question, x_pos_prop], np.array(y)

df_train = df[df.query_id.isin(trainIds)]
bpr = CLF()
x_train, y_train = bpr.generate_train_data(df_train)
bestNDCG = 0
bestRes = {}
for i in range(10):
    test_num = 0
    history = bpr.model.fit(x_train, y_train, verbose=0)
    res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
    for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
        if row['entity'] not in entityInWiki:
                continue
        test_num += 1
        qrels = qrel[str(row['query_id'])]
        cand_properties = type2prop[row['entityType']]
#         cand_properties = list(prop2MSvec.keys())
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

        rank = {}
        for p in cand_properties:
            if row['query_id'] in qid2questions2MSvec:
                
                
                score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), pad_sequences(qid2questions2MSvec[row['query_id']], maxlen=MAX_QUESTION_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            else:    
                score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), pad_sequences([[]], maxlen=MAX_QUESTION_LENGHT), np.array([prop2MSvec[p]])])[0][0]
#             score = bpr.model.predict([pad_sequences([[qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            rank[p] = score
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        for key in res:
            res[key].append(our[key])
            
    ndcg = np.mean(res["ndcg@15"])
    if ndcg > bestNDCG:
        bestRes = res
        bestNDCG = ndcg

    for key in res:
        print(np.mean(res[key]), end="\t")
    print()
# runDocEval(qid2MSvec, prop2MSvec)

In [151]:
np.random.rand(100)

array([0.03433255, 0.44975244, 0.53311028, 0.35410887, 0.99748882,
       0.73896041, 0.05642281, 0.07724881, 0.30524471, 0.29512638,
       0.97062959, 0.718337  , 0.902602  , 0.47875612, 0.88266367,
       0.81159707, 0.32974246, 0.95067312, 0.00537919, 0.64954441,
       0.30603224, 0.72527115, 0.53430609, 0.13414405, 0.53767863,
       0.47238097, 0.73724248, 0.46319916, 0.15872898, 0.26946032,
       0.00593328, 0.17845321, 0.98740532, 0.91324818, 0.03026555,
       0.25765835, 0.04933299, 0.34362212, 0.35729627, 0.1945712 ,
       0.66704162, 0.09759326, 0.79909578, 0.38164191, 0.02747136,
       0.96399566, 0.645902  , 0.84524526, 0.65987698, 0.44199329,
       0.96531829, 0.65426018, 0.32554636, 0.25989493, 0.14494618,
       0.43417079, 0.30338033, 0.97656624, 0.70829141, 0.10266963,
       0.93817736, 0.83154394, 0.07704345, 0.03940803, 0.31647717,
       0.60313842, 0.84947373, 0.35856523, 0.43557343, 0.50307816,
       0.73536156, 0.36752177, 0.59516322, 0.7562519 , 0.04427

In [83]:
# CLF with Question MS vectors
max_questions = np.max([len(qid2questions2MSvec[i]) for i in qid2questions2MSvec])
MAX_SEQUENCE_LENGHT = int(np.max([len(qid2MSvec[i]) for i in qid2MSvec])) + max_questions + 1
# def runDocEval(qid2MSvec, prop2MSvec):
class CLF():
    def __init__(self):

        self.queryInput = Input(shape=(MAX_SEQUENCE_LENGHT, 100,))
        self.propPosInput = Input(shape=(100,))
        lstm = LSTM(100)

        queryEmbeddingLayer = Dense(100, name="uEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

#         qEmb = queryEmbeddingLayer(lstm(self.queryInput))
        qEmb = queryEmbeddingLayer(self.queryInput)
        pEmb = propEmbeddingLayer(self.propPosInput)

        dense = Dense(1, activation="sigmoid")

        pred = Multiply()([qEmb, pEmb])
        
        self.pred = dense(GlobalMaxPooling1D()(pred))
#         self.pred = dense(pred)

        self.model = Model(inputs=[self.queryInput, self.propPosInput], outputs=self.pred)
        self.model.compile(optimizer="adam", loss="binary_crossentropy")

    def generate_train_data(self, df):
        x_query, x_pos_prop, y = [], [], []
        for name, group in df.groupby("query_id"):
            cand_pos_prop = group.property.tolist()
            rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()
            for idx, row in group.iterrows():
                if name in qid2questions2MSvec:
                    x_query.append(qid2MSvec[name] + qid2questions2MSvec[name] + [qid2query2MSvec[name]])
                else:
                    x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[row['property']])
                y.append(1)
        
            cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
#             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
            
            count = 0
            while count != len(group):
                for neg_prop in cand_neg_prop:
                    if name in qid2questions2MSvec:
                        x_query.append(qid2MSvec[name] + qid2questions2MSvec[name] + [qid2query2MSvec[name]])
                    else:
                        x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
    #                 x_query.append([qid2query2MSvec[name]])
                    x_pos_prop.append(prop2MSvec[neg_prop])
                    y.append(0)
                    count += 1
                    if count == len(group):
                        break

        x_query = pad_sequences(x_query,dtype='float32', maxlen=MAX_SEQUENCE_LENGHT)
        x_pos_prop = np.array(x_pos_prop)
        return [x_query, x_pos_prop], np.array(y)

df_train = df[df.query_id.isin(trainIds)]
bpr = CLF()
x_train, y_train = bpr.generate_train_data(df_train)
bestNDCG = 0
bestRes = {}
for i in range(10):
    test_num = 0
    history = bpr.model.fit(x_train, y_train, verbose=0)
    res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
    for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
        if row['entity'] not in entityInWiki:
                continue
        test_num += 1
        qrels = qrel[str(row['query_id'])]
        cand_properties = type2prop[row['entityType']]
#         cand_properties = list(prop2MSvec.keys())
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

        rank = {}
        for p in cand_properties:
            if row['query_id'] in qid2questions2MSvec:
                score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']]+ qid2questions2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            else:    
                score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
#             score = bpr.model.predict([pad_sequences([[qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            rank[p] = score
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        for key in res:
            res[key].append(our[key])
            
    ndcg = np.mean(res["ndcg@15"])
    if ndcg > bestNDCG:
        bestRes = res
        bestNDCG = ndcg

    for key in res:
        print(np.mean(res[key]), end="\t")
    print()
# runDocEval(qid2MSvec, prop2MSvec)

0.671698113207547	0.6366876310272537	0.6203354297693919	0.22388223817060196	0.4133358055643235	0.57834138494253	0.47090433295647216	0.5183526025857245	0.5728543191502038	
0.709433962264151	0.6649895178197064	0.6341719077568133	0.2331357750679849	0.4283894973210938	0.5894220754484205	0.49540236929097214	0.5444787522211142	0.5870978820484891	
0.7245283018867924	0.661215932914046	0.629140461215933	0.23857754092069766	0.4283239182298706	0.5849860094893391	0.5176873486004275	0.555101694843192	0.592600191636385	
0.7169811320754715	0.6819706498951781	0.6429769392033543	0.2317280128695807	0.43903134497503327	0.5950146069149806	0.5024859418527098	0.5527983263296364	0.5912588846363841	
0.6830188679245284	0.6555555555555554	0.6266247379454928	0.2258671590431623	0.42203337826636494	0.581666006122312	0.4961853233090351	0.5453267137717537	0.5871258145110986	
0.7358490566037734	0.6649895178197064	0.6253668763102724	0.24180761878203316	0.42761346905134506	0.58336777066551	0.5189738054601633	0.55220646

In [40]:
# Most recent and best performing model
MAX_SEQUENCE_LENGHT = int(np.max([len(qid2MSvec[i]) for i in qid2MSvec])) + 1
# def runDocEval(qid2MSvec, prop2MSvec):
class CLF():
    def __init__(self):

        self.queryInput = Input(shape=(MAX_SEQUENCE_LENGHT, 100,))
        self.propPosInput = Input(shape=(100,))
        lstm = LSTM(100)

        queryEmbeddingLayer = Dense(100, name="uEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

#         qEmb = queryEmbeddingLayer(lstm(self.queryInput))
        qEmb = queryEmbeddingLayer(self.queryInput)
        pEmb = propEmbeddingLayer(self.propPosInput)

        dense = Dense(1, activation="sigmoid")

        pred = Multiply()([qEmb, pEmb])
        
        self.pred = dense(GlobalMaxPooling1D()(pred))
#         self.pred = dense(pred)

        self.model = Model(inputs=[self.queryInput, self.propPosInput], outputs=self.pred)
        self.model.compile(optimizer="adam", loss="binary_crossentropy")

    def generate_train_data(self, df):
        x_query, x_pos_prop, y = [], [], []
        for name, group in df.groupby("query_id"):
            cand_pos_prop = group.property.tolist()
            rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()
            for idx, row in group.iterrows():
                x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[row['property']])
                y.append(1)

            cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
#             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
            for neg_prop in cand_neg_prop:
                x_query.append(qid2MSvec[name] + [qid2query2MSvec[name]])
#                 x_query.append([qid2query2MSvec[name]])
                x_pos_prop.append(prop2MSvec[neg_prop])
                y.append(0)

        x_query = pad_sequences(x_query,dtype='float32', maxlen=MAX_SEQUENCE_LENGHT)
        x_pos_prop = np.array(x_pos_prop)
        return [x_query, x_pos_prop], np.array(y)

df_train = df[df.query_id.isin(trainIds)]
bpr = CLF()
x_train, y_train = bpr.generate_train_data(df_train)
for i in range(10):
    test_num = 0
    history = bpr.model.fit(x_train, y_train, verbose=0)
    res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
    for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
        if row['entity'] not in entityInWiki:
                continue
        test_num += 1
        qrels = qrel[str(row['query_id'])]
        cand_properties = type2prop[row['entityType']]
#         cand_properties = list(prop2MSvec.keys())
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))
#         cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

        rank = {}
        for p in cand_properties:
            score = bpr.model.predict([pad_sequences([qid2MSvec[row['query_id']] + [qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
#             score = bpr.model.predict([pad_sequences([[qid2query2MSvec[row['query_id']]]], dtype='float32', maxlen=MAX_SEQUENCE_LENGHT), np.array([prop2MSvec[p]])])[0][0]
            rank[p] = score
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        for key in res:
            res[key].append(our[key])
    ndcg = np.mean(res["ndcg@15"])
    if ndcg > bestNDCG:
        bestRes = res
        bestNDCG = ndcg

for key in bestRes:
    print(np.mean(bestRes[key]), end="\t")
print()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



NameError: name 'bestNDCG' is not defined

In [83]:
def runEval(qid2vec, prop2vec):
    class CLF():
        def __init__(self):

            self.queryInput = Input(shape=(100,))
            self.propPosInput = Input(shape=(100,))

            queryEmbeddingLayer = Dense(100, name="uEmb")
            propEmbeddingLayer = Dense(100, name="iEmb")

            qEmb = queryEmbeddingLayer(self.queryInput)
            pEmb = propEmbeddingLayer(self.propPosInput)

            dense = Dense(1, activation="sigmoid")

            pred = Multiply()([qEmb, pEmb])
            self.pred = dense(pred)

            self.model = Model(inputs=[self.queryInput, self.propPosInput], outputs=self.pred)
            self.model.compile(optimizer="adam", loss="binary_crossentropy")
    #         self.predictor = Model([self.queryInput, self.propPosInput], [pDot])
    #         self.predictor = Model([self.entityInput, self.entityTypeInput, self.actionInput, self.propPosInput], [pDot])
        def generate_train_data(self, df):
            x_query, x_entity, x_type, x_action, x_pos_prop, x_neg_prop, y = [], [], [], [], [], [], []
            for name, group in df.groupby("query_id"):
                cand_pos_prop = group.property.tolist()
                rele2prop = group[['rele_label', 'property']].groupby("rele_label")["property"].apply(list).to_dict()

                for idx, row in group.iterrows():

                    x_query.append(qid2query2MSvec[name])
                    x_pos_prop.append(prop2vec[row['property']])
                    y.append(1)

                cand_neg_prop = list(set.difference(set(type2prop[row['entityType']]), set(cand_pos_prop)))
    #             cand_neg_prop = list(set.difference(set(df.property.unique()), set(cand_pos_prop)))
                for neg_prop in cand_neg_prop:
                    x_query.append(qid2query2MSvec[name])
                    x_pos_prop.append(prop2vec[neg_prop])
                    y.append(0)

            x_query = np.array(x_query)
            x_entity = np.array(x_entity)
            x_type = np.array(x_type)
            x_action = np.array(x_action)
            x_pos_prop = np.array(x_pos_prop)
            x_neg_prop = np.array(x_neg_prop)
    #         return [x_entity, x_type, x_action, x_pos_prop, x_neg_prop], np.ones(len(x_query))
            return [x_query, x_pos_prop], np.array(y)

    df_train = df[df.query_id.isin(trainIds)]
    bpr = CLF()
    bestNDCG = 0
    for i in range(10):
        x_train, y_train = bpr.generate_train_data(df_train)
        res = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg"] for j in [5, 10 ,15]}
        history = bpr.model.fit(x_train, y_train, verbose=0)
        for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
            if row['query_id'] not in testIds:
                continue
            if row['entity'] not in entityInWiki:
                continue
            qrels = qrel[str(row['query_id'])]
            cand_properties = type2prop[row['entityType']]
#             cand_properties = list(set(type2prop[row['entityType']].tolist() + list(qrels.keys())))

            rank = {}
            for p in cand_properties:
                score = bpr.model.predict([[qid2query2MSvec[row['query_id']]], [prop2vec[p]]])[0][0]
                rank[p] = score
            rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
            our = evaluate(qrels, rank)
#             res.append(our)
            
            for key in res:
                res[key].append(our[key])
        ndcg = np.mean(res["ndcg@15"])
        if ndcg > bestNDCG:
            bestRes = res
            bestNDCG = ndcg
    for key in bestRes:
        print(np.mean(bestRes[key]), end="\t")
    print()
runEval(qid2query2MSvec, prop2MSvec)

NameError: name 'qid2query2MSvec' is not defined

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
for idx, row in df[['query_id', 'entity', 'action', 'entityType', 'query']].drop_duplicates().iterrows():
    if row['query_id'] not in testIds:
        continue
    if row['entity'] not in entityInWiki:
        continue
    qrels = qrel[str(row['query_id'])]
    cand_properties = type2prop[row['entityType']]

    rank = {}
    for p in cand_properties:
#         score = cosine_similarity([qid2query2MSvec[row['query_id']]], [prop2MSvec[p]])[0][0]
        score = cosine_similarity(qid2MSvec[row['query_id']], [prop2MSvec[p]])[0][0]
        rank[p] = score
    rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
    our = evaluate(qrels, rank)
#             res.append(our)

    for key in res:
        res[key].append(our[key])

for key in res:
    print(np.mean(res[key]), end="\t")


NameError: name 'prop2MSvec' is not defined

In [79]:
res, res2, res3 = [], [], []
for fold in range(5):
    _df = pd.read_csv("data/cedr/test%d.tsv" %fold, sep="\t", usecols=[0], names=["qid"])
    testIds = _df.qid.unique()
    prop2popularity = df.groupby("property").size().to_dict()

    
    for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue
#         if row['entity'] not in entityInWiki:
#             continue
        qrels = qrel[str(row['query_id'])]
        cand_properties = type2prop[row['entityType']]

        rank = {}
        for p in cand_properties:
            rank[p] = 1
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        
    #     our = evaluate(qrels, cand_properties)
        base = evaluate(qrels, list(run["1"][str(row["query_id"])].keys()))
        res.append(our)
        res2.append(base)
    #     print(key, np.mean(res2[key]))
    
        rank = {}
        for p in cand_properties:
            rank[p] = prop2popularity[p] if p in prop2popularity else 0
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        our = evaluate(qrels, rank)
        res3.append(our)

keys = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg", "nerr"] for j in [5, 10 ,15, 20]}
keys["rp"] = []
for r in [res, res2, res3]:
    for k in keys:
        print(np.mean([i[k] for i in r]), end="\t")
    print()

0.6101604278074866	0.5948900772430183	0.5896316102198454	0.5574123588829472	0.20284161831080866	0.3989336998954003	0.5614833881840439	0.6809485818203794	0.40964763205666543	0.47519438511873113	0.5273372667425006	0.5693692367007145	0.5358990143229291	0.5609055996150769	0.563725013589058	0.5643480165469257	0.5830465112235345	
0.6850267379679144	0.6479946524064172	0.6486185383244206	0.6317289172916564	0.23021496830630514	0.41787634293956993	0.5984617287476647	0.7509116055739267	0.476448623780293	0.5278082500975781	0.5786404600253108	0.6365679457993964	0.5914453777924802	0.6107507241365906	0.6127116370974913	0.6129443731413323	0.6199307191531469	
0.5379679144385027	0.5117349970291147	0.543285799168152	0.5485888294711824	0.19291116746854142	0.34842390242658505	0.5183212550711478	0.6702678707001852	0.37389565453132	0.42318352491460753	0.48473882856970074	0.5464452199439942	0.5017111667854726	0.5313927487094723	0.5367587117675158	0.537517480835144	0.5496962850794768	


# CEDR

In [59]:
# create property index
prop2pid = {p:i for i, p in enumerate(dfp.property.unique())}
df["pid"] = [prop2pid[i] for i in df['property']]
type2allprop = dfp.groupby("entityType")['property'].unique().to_dict()

In [ ]:
# " [SEP] ".join([title, overview, sectionLabel, headline, text])

In [109]:
for idx, row in df.drop_duplicates("query_id").iterrows():
    with open("data/cedr/entity.tsv", 'a') as fd:
        fd.write("query\t"+str(row['query_id'])+"\t"+ row['entity'] +"\n")


In [274]:
df_bm25 = pd.read_csv("data/cedr/query-title-bm25.tsv", names=["q", "qid", "text"], sep="\t")
# for col in ["overview", "sectionLabel", "headline", "text"]:
for col in ["question-qw"]:
    for idx, row in df.drop_duplicates("query_id").iterrows():
    #     with open("data/cedr/query.tsv", 'a') as fd:
    #         fd.write("query\t"+str(row['query_id'])+"\t"+row["entity"]+" "+row["action"]+"\n")
        with open("data/cedr/%s.tsv" % col, 'a') as fd:
            query = row['action'] + " " + row['entity']
            query = query.replace("\'", "")
#             tokenized_query = query.split(" ")
    #         doc = bm25.get_top_n(tokenized_query, corpus, n=10)
    #         titles = [doc2title[i] for i in doc]
    #         overview = doc[0].split(" [SEP] ")[4]
    #         text = " ".join(df_wiki[df_wiki.title == titles[0]].text.unique().tolist())
    #         print(overview)
    #     break
    #         text = " ".join(df_wiki[df_wiki.title == titles[0]].text.tolist())
    #         fd.write("query\t"+str(row['query_id'])+"\t"+query+" "+titles[0]+ " " + text +"\n")
    #         fd.write("query\t"+str(row['query_id'])+"\t"+query+" "+titles[0] +"\n")
    
#     questions
            title = [df_bm25[df_bm25.text.str.contains(query[:10])].text.values[0].replace(query+" ", "")]
            try:
                lines = open("data/questions/wiki/%s.csv" % title[0].replace(" ", "_")).read().splitlines()[1:]
            except:
#                 fd.write("query\t"+str(row['query_id'])+"\t"+query+" "+title[0] +"\n")
#                 fd.write("query\t"+str(row['query_id'])+"\t"+query+"\n")
                fd.write("question\t"+str(row['query_id'])+"\t"+query+"\n")
                continue
            questions = []
            for i in lines[1:]:
                questions.extend(i.split(";"))
            questions = " ".join(list(set(questions)))
            if questions == "":
                fd.write("question\t"+str(row['query_id'])+"\t"+query+"\n")
            else:
#             fd.write("query\t"+str(row['query_id'])+"\t"+query+" "+title[0]+ " " + questions +"\n")
#             fd.write("query\t"+str(row['query_id'])+"\t"+query+" " + questions +"\n")
                fd.write("question\t"+str(row['query_id'])+"\t"+ questions +"\n")
            
#         break
            
#             text = " ".join(df_wiki[df_wiki.title == titles[0]][col].unique().tolist())
#             fd.write("query\t"+str(row['query_id'])+"\t"+query+" "+titles[0]+ " " + text +"\n")

    #         if row['query_id'] not in qid2wiki:
    #             fd.write("query\t"+str(row['query_id'])+"\t"+query+"\n")
    #             doc = bm25.get_top_n(tokenized_query, corpus, n=5)
    #             title = [doc2title[i] for i in doc]
    #             print(row['action'], ",", row['entity'], "\t", title)
    #             continue
    #         BERT
    #         titles = dfw[dfw.title.str.contains(row['entity'])].title.unique().tolist()
    #         rank = {}
    #         for title in titles:
    #             rank[title] = bertSim(query, title)
    #         best_title = sorted(rank.items(), key=lambda x: x[1])[-1][0]
    #         text = " ".join(df_wiki[df_wiki.title == best_title].text.tolist())
    #         fd.write("query\t"+str(row['query_id'])+"\t"+query+" "+best_title+ " "+ text +"\n")

  

In [719]:
df_bm25 = pd.read_csv("data/cedr/query-title-bm25.tsv", names=["q", "qid", "text"], sep="\t")
for col in ["overview", "headline", "sectionLabel","text"]:
    for idx, row in df.drop_duplicates("query_id").iterrows():
        with open("data/cedr/wiki-%s-bm25.tsv" % col, 'a') as fd:
            query = row['action'] + " " + row['entity']
            query = query.replace("\'", "")
            title = df_bm25[df_bm25.text.str.contains(query[:10])].text.values[0].replace(query+" ", "")
            text = " ".join(df_wiki[df_wiki.title == title][col].unique().tolist())
            if text == "steps" or text == "":
                fd.write("wiki\t"+str(row['query_id'])+"\t"+ query +"\n")                
            else:
                fd.write("wiki\t"+str(row['query_id'])+"\t"+ text +"\n")

In [302]:
import glob
# qid2questions2MSvec = {}
for file in glob.glob("akgBM25/*"):
    title = file.split("/")[1].replace("_", " ").replace(".csv", "")
    lines = open(file).read().splitlines()
    questions = []
    for i in lines[1:]:
        questions.extend(i.split(";"))
    print(bestTitle2qid[title], title, questions, df[df.query_id == bestTitle2qid[title]])
    
    
    
    
    break
#     title2questions[title] = list(set(questions))
#     data = getVectors([title] + list(set(questions)) )
#     qid2questions2MSvec[bestTitle2qid[title]] = list(data.values())
# qid2questions2MSvec = loadDict("qid2questions2MSvec")

222 spray lacquer ['What does spray lacquer do?', 'Does lacquer make wood waterproof?', 'What does spray lacquer do?', 'Does lacquer seal wood?', 'What is the difference between varnish and lacquer?', 'Is lacquer good for outdoors?', 'What is the difference between varnish and lacquer?', 'Does lacquer turn yellow?', 'What is the difference between varnish and lacquer?', "What's better lacquer or polyurethane?", 'How long does it take for spray lacquer to dry?', 'How long does lacquer last?', 'How long does it take for spray lacquer to dry?', 'Should I sand between coats of lacquer?', 'How long does it take for spray lacquer to dry?', 'Does lacquer scratch easily?', 'What is the difference between lacquer and polyurethane finish?', 'What is the best lacquer for wood?', 'What is the difference between lacquer and polyurethane finish?', 'Is lacquer thicker than polyurethane?', 'What is the difference between lacquer and polyurethane finish?', 'Can you brush on lacquer?']       query_id   

In [294]:
# for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
for name, group in df.groupby("query_id"):
    entity = group.entity.unique()[0]
    action = group.action.unique()[0]
    entityType = group.entityType.unique()[0]
    properties = " ".join(group.property.unique())
    query = entity + " " + action + " " + entityType
    doc = bm25.get_top_n(tokenizer.tokenize(query), corpus, n=10)
#     print(tokenizer.tokenize(query), group.entityType.unique(), properties)
    print(query)
    print(doc[0].split("\t")[0])
    print()
#     for d in doc:
#         print(d.split("\t")[0])
    
    
#     data = getVectors(doc)

benchmarking have thing action
overclock graphics card

bodybuilding take thing action
begin bodybuilding

brainstorming brainstorm thing action
develop alternative perspectives decision making

chain smoking start thing action
quit smoking using allen carr book

funding get funding thing action
get venture capital investment

hair coloring bleach hair back to brown thing action
ombre hair

hit and run stop at the scene of an accident thing action
know whether call police after car accident

language acquisition suggest a foreign language thats easy to learn thing action
teach adults foreign language

mediation help you come up with possible solutions thing action
prepare child custody mediation

pull up do any aerobic physical activity thing action
improve your study routine exercise

spoofing attack know who do the spoof  thing action
make funny movie spoof

spray painting use on fabric thing action
spray paint your sofa

translation know japanese recipe ingredient translation thing 

little league baseball join thing organization sportsorganization
become baseball writer

los angeles lakers win thing organization sportsorganization
be good orlando magic fan

major league baseball announce retirement thing organization sportsorganization
become baseball writer

maroon 5 perform ballad thing organization performinggroup musicgroup
access shared folders windows 7

maybelline buy makeup thing organization company
buy beauty products budget

miami heat win the championship thing organization sportsorganization
be good orlando magic fan

motorola unlock motorola c975 thing organization corporation
remove motorola subsidy password message

nvidia take complaint thing organization company
update nvidia drivers

people for the ethical treatment of animals treat animals thing organization ngo
help stop cruelty towards animals

pga tour win tournament thing organization sportsorganization
win your ncaa basketball tournament pool

singapore airlines operate route thing organiz

KeyboardInterrupt: 

In [303]:
df[["entity", "action", "entityType"]].drop_duplicates().to_csv("tmp.csv")

In [120]:
for name, group in df.groupby("query_id"):
    query = group.entity.iloc[0] + " " + group.action.iloc[0] + " " + group.entityType.iloc[0] # .replace("thing ","").replace("thing","")
    query = group.entityType.iloc[0] + " " + group.action.iloc[0]# .replace("thing ","").replace("thing","")
    with open("data/cedr/type-action.tsv", 'a') as fd:
        fd.write("query\t"+str(name)+"\t"+query +"\n")

In [256]:
df.groupby("entityType").size()

entityType
thing                                                               5
thing action                                                      139
thing creativework                                                 22
thing creativework book                                            12
thing creativework movie                                          379
thing creativework musicrecording                                  42
thing creativework tvseries                                        61
thing event                                                       157
thing event educationevent                                         22
thing event sportsevent                                            19
thing intangible                                                   59
thing intangible computerlanguage                                  11
thing intangible language                                           4
thing intangible service                                           38
thing int

In [253]:
type2prop['thing creativework movie']

array(['recordedAt', 'musicBy', 'character', 'workExample',
       'educationalUse', 'releasedEvent', 'actor', 'author', 'version',
       'isFamilyFriendly', 'material', 'publisher', 'producer',
       'accountablePerson', 'commentCount', 'alternativeHeadline',
       'headline', 'timeRequired', 'potentialAction', 'isPartOf',
       'exampleOfWork', 'hasPart', 'isBasedOn', 'position', 'video',
       'accessibilityHazard', 'mentions', 'provider', 'name', 'audio',
       'copyrightYear', 'contentRating', 'creator', 'award', 'director',
       'productionCompany', 'genre', 'schemaVersion', 'trailer',
       'publication', 'audience', 'countryOfOrigin', 'comment',
       'alternateName', 'inLanguage', 'offers', 'identifier',
       'publishingPrinciples', 'text', 'contentLocation',
       'locationCreated', 'translator', 'review', 'image'], dtype=object)

In [251]:
df.groupby("entityType").size()

entityType
thing                                                               5
thing action                                                      139
thing creativework                                                 22
thing creativework book                                            12
thing creativework movie                                          379
thing creativework musicrecording                                  42
thing creativework tvseries                                        61
thing event                                                       157
thing event educationevent                                         22
thing event sportsevent                                            19
thing intangible                                                   59
thing intangible computerlanguage                                  11
thing intangible language                                           4
thing intangible service                                           38
thing int

In [282]:
for key in prop2pid:
    with open("data/cedr/doc.tsv", 'a') as fd:
        fd.write("doc\t"+str(prop2pid[key])+"\t"+key+"\n")


In [278]:
for idx, row in df.iterrows():
    with open("data/cedr/qrel.tsv", 'a') as fd:
        fd.write(str(row["query_id"]) + "\t0\t"+str(row["pid"])+"\t"+str(row["rele_label"])+"\n")

In [61]:
#5fold data generation
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold

foldNum = 5
trainId5fold = {i:[] for i in range(foldNum)}
validateId5fold = {i:[] for i in range(foldNum)}
testId5fold = {i:[] for i in range(foldNum)}
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() >= foldNum:
        ids = np.array(group.query_id.unique())
        
        kf = KFold(n_splits=foldNum)
        kf.get_n_splits(ids)
        for idx, ele in enumerate(kf.split(ids)):
            train, test = ids[ele[0]], ids[ele[1]]
            train, val, _, _ = train_test_split(train, train, test_size=0.2)
            trainId5fold[idx].extend(train)
            validateId5fold[idx].extend(val)
            testId5fold[idx].extend(test)
    else:
        ids = list(group.query_id.unique())
        for i in range(foldNum):
            trainId5fold[i].extend(ids)

def writeRunFile(filename, _df):
    for name, group in _df.groupby("query_id"):
        for idx, row in enumerate(group.iterrows()):
            with open(filename, 'a') as fd:
                fd.write(str(row[1]['query_id']) +"\tQ0\t" + str(row[1]['pid']) + "\t" + str(idx+1) + "\t" + str(row[1]['rele_label']) + "\trun\n")
                
for fold in range(foldNum):
    for idx, group in df[df.query_id.isin(trainId5fold[fold])].groupby("query_id"):
        pos = group['property'].tolist()
        neg = list(set.difference(set(type2allprop[group.iloc[0]['entityType']]), set(pos)))

        for idx, row in group.iterrows():
            with open("data/cedr/train%d.tsv" % fold, 'a') as fd:
                fd.write(str(row["query_id"]) + "\t"+str(row["pid"])+"\n")
        for n in neg:
            with open("data/cedr/train%d.tsv" % fold, 'a') as fd:
                fd.write(str(row["query_id"]) + "\t"+str(prop2pid[n])+"\n")
    
    for idx, group in df[df.query_id.isin(validateId5fold[fold])].groupby("query_id"):
        pos = group['property'].tolist()
        neg = list(set.difference(set(type2allprop[group.iloc[0]['entityType']]), set(pos)))

        lines = {}
        for idx, row in group.iterrows():
            lines[str(row["query_id"]) + "\tQ0\t"+str(prop2pid[row['property']])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']

        for n in neg:
            lines[str(row["query_id"]) + "\tQ0\t"+str(prop2pid[n])+"\t0\t0\trun\n"] = 0
        lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
        for l in lines:
            with open("data/cedr/valid%d.tsv" % fold, 'a') as fd:
                fd.write(l)
                
    for idx, group in df[df.query_id.isin(testId5fold[fold])].groupby("query_id"):
        pos = group['property'].tolist()
        neg = list(set.difference(set(type2allprop[group.iloc[0]['entityType']]), set(pos)))

        lines = {}
        for idx, row in group.iterrows():
            lines[str(row["query_id"]) + "\tQ0\t"+str(prop2pid[row['property']])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']

        for n in neg:
            lines[str(row["query_id"]) + "\tQ0\t"+str(prop2pid[n])+"\t0\t0\trun\n"] = 0
        lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
        for l in lines:
            with open("data/cedr/test%d.tsv" % fold, 'a') as fd:
                fd.write(l)
    

In [279]:
# split train and test
for idx, group in df[df.query_id.isin(trainIds)].groupby("query_id"):
    pos = group['property'].tolist()
    neg = list(set.difference(set(type2allprop[group.iloc[0]['entityType']]), set(pos)))

    for idx, row in group.iterrows():
        with open("data/cedr/train.tsv", 'a') as fd:
            fd.write(str(row["query_id"]) + "\t"+str(row["pid"])+"\n")
    for n in neg:
        with open("data/cedr/train.tsv", 'a') as fd:
            fd.write(str(row["query_id"]) + "\t"+str(prop2pid[n])+"\n")

In [280]:
for idx, group in df[df.query_id.isin(testIds)].groupby("query_id"):
    pos = group['property'].tolist()
    neg = list(set.difference(set(type2allprop[group.iloc[0]['entityType']]), set(pos)))

    lines = {}
    for idx, row in group.iterrows():
#         with open("data/cedr/test.run", 'a') as fd:
#             fd.write(str(row["query_id"]) + "\tQ0\t"+str(prop2pid[n])+"\t0\t" +str(row['rele_label'])+"\trun\n")
        lines[str(row["query_id"]) + "\tQ0\t"+str(prop2pid[row['property']])+"\t0\t" +str(row['rele_label'])+"\trun\n"] = row['rele_label']
            
    for n in neg:
#         with open("data/cedr/test.run", 'a') as fd:
        lines[str(row["query_id"]) + "\tQ0\t"+str(prop2pid[n])+"\t0\t0\trun\n"] = 0
    lines = [i[0] for i in sorted(lines.items(), key=lambda x: x[1], reverse=True)]
    for l in lines:
        with open("data/cedr/test.tsv", 'a') as fd:
            fd.write(l)

In [268]:
len(df[df.query_id == 71]), len(type2allprop2[df[df.query_id == 71].iloc[0].entityType])

(23, 27)

In [264]:
c, c1, c2 = 0, 0, 0
for idx, group in df.groupby("query_id"):
    pos = group.property.tolist()
    neg1 = list(set.difference(set(type2allprop[group.iloc[0]['entityType']]), set(pos)))
    neg2 = list(set.difference(set(type2allprop2[group.iloc[0]['entityType']]), set(pos)))
    if len(neg1) == 0:
        c1 += 1
    if len(neg2) == 0:
        c2 += 1
    c += 1
print(c, c1, c2)

229 20 0


In [21]:
from modeling import BertRanker
from Data import _pad_crop, _mask
from modeling_util import SimmatModule

In [22]:
br = BertRanker()

In [42]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [46]:
def get(query, doc):
    _query_token = br.tokenize(query)
    _doc_token = br.tokenize(doc)
    query_token = _pad_crop([_query_token], 5)
    doc_token = _pad_crop([_doc_token], 5)
    query_mask = _mask([_query_token], 5)
    doc_mask = _mask([_doc_token], 5)
    return br.encode_bert(query_token, query_mask, doc_token, doc_mask)

get("have benchmarking", "startTime")[1][12].shape

torch.Size([1, 5, 768])

In [110]:
from scipy.spatial.distance import cosine
def bertSim(query, doc):
    QLEN = DLEN = 20
    query = br.tokenize(query)
    doc = br.tokenize(doc)
    query_tok = _pad_crop([query], QLEN)
    doc_tok = _pad_crop([doc], DLEN)
    query_mask = _mask([query], QLEN)
    doc_mask =_mask([doc], DLEN)
    c, q, d = br.encode_bert(query_tok, query_mask, doc_tok, doc_mask)
    return 1 - cosine(torch.mean(q[-1][0], dim=0).detach().numpy(), torch.mean(d[-1][0], dim=0).detach().numpy())


In [71]:
for idx, row in df.iterrows():
    print(row['query_id'])
    bertSim(row['entity'], row['property']), row['rele_label']
    print(1 - cosine()
)
    break

1
13


In [63]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=

In [281]:
def getBertVec(text):
    
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
        
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        
#       second to last
        token_vecs = encoded_layers[0][0]
#         token_vecs = torch.squeeze(torch.sum(torch.stack(emb), dim=0), dim=0)

        sentence_embedding = torch.mean(token_vecs, dim=0)

        return sentence_embedding.detach().numpy()
def bertSim2(query, doc):
    return 1 - cosine(getBertVec(query), getBertVec(doc))

# for idx, row in df.iterrows():
#     print(row['query_id'])
#     print(bertSim2(row['action'] +" " +row['entity'], row['property']), row['rele_label'])

#     for col in ["title", "overview", "sectionLabel", "headline", "text"]:
for col in [3,4]:
    res, res2, res3 = [], [], []
    for fold in range(5):
        _df = pd.read_csv("data/cedr/test%d.tsv" %fold, sep="\t", usecols=[0], names=["qid"])
        testIds = _df.qid.unique()

        for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
            if row['query_id'] not in testIds:
                continue
#             print(fold, row['query_id'])
            qrels = qrel[str(row['query_id'])]
            cand_properties = type2prop[row['entityType']]

            rank = {}
            doc = bm25.get_top_n(tokenizer.tokenize(row['action'] +" " +row['entity']), corpus, n=1)[0].split("\t")[col][:500]

            for p in cand_properties:
    #             rank[p] = bertSim2(row['action'] +" " +row['entity'], p)
                rank[p] = bertSim2(doc, p)
    #             print(rank[p])
            rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
            our = evaluate(qrels, rank)

            res.append(our)

    keys = {"%s@%d" %( i,j): [] for i in ["p", "r", "ndcg", "nerr"] for j in [5, 10 ,15, 20]}
    keys["rp"] = []
    for r in [res]:
        for k in keys:
            print(np.mean([i[k] for i in r]), end="\t")


0.5887700534759358	0.5579916815210934	0.5498811645870469	0.5370915032679738	0.2023824341397186	0.37383273408613293	0.5205524147829946	0.6563343661279278	0.41748150210732166	0.46722664435583194	0.5077280078477923	0.5583809990552392	0.5483985055151065	0.5730014618779197	0.57668781331345	0.5775744590750268	0.5401313764702022	0.5732620320855615	0.5561200237670827	0.5415032679738562	0.5312091503267974	0.19533384738903942	0.373239288499416	0.5152811387331357	0.6512255028340639	0.40852621914075843	0.46013109212750586	0.4984495450441254	0.549658459665293	0.5485529136943489	0.5740434585853085	0.578324970616467	0.5791509041302285	0.5332822412411633	

In [285]:
df.groupby("entity")["property"].unique()

entity
abraham lincoln          [children, gender, memberOf, hasOfferCatalog, ...
acapulco                 [image, logo, event, telephone, review, contai...
acupuncture              [relevantSpeciality, anatomicalSystem, medical...
adidas                   [globalLocationNumber, name, brand, makesOffer...
alan turing              [name, deathPlace, potentialAction, knows, hei...
all about my mother      [recordedAt, musicBy, character, workExample, ...
angola                   [geo, description, telephone, containsPlace, a...
anti-gravity             [potentialAction, additionalType, description,...
aol                      [location, numberOfEmployees, description, add...
apgar score              [image, potentialAction, description, code, st...
apple cider              [isSimilarTo, productionDate, logo, purchaseDa...
aristotle                [workLocation, relatedTo, image, nationality, ...
asthma                   [associatedAnatomy, primaryPrevention, descrip...
automotive battery

In [271]:
doc

'overclock graphics card'

In [131]:
def getBertVec(text):
    
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
        
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)
        print(encoded_layers[0] + encoded_layers[1])
        
#       sum
#         token_vecs = torch.sum(encoded_layers, dim=0)[0]
#         token_vecs = encoded_layers[-1][0]
        token_vecs = torch.sum(torch.stack(emb), dim=0)
    
#         print(token_vecs.size())
        sentence_embedding = torch.mean(token_vecs, dim=0)

        return sentence_embedding.detach().numpy(), encoded_layers
# _, emb = getBertVec("test")

tensor([[[ 0.1307, -0.2163, -0.2279,  ...,  0.0138,  0.1159,  0.0976],
         [ 2.2972, -0.5218, -0.3770,  ...,  0.7855,  0.5923, -4.9415],
         [-0.7612,  0.3292,  0.2475,  ...,  0.3704,  1.8085,  0.0186]]])


In [132]:
torch.stack(emb).size()

torch.Size([12, 1, 3, 768])

In [141]:
emb[-1].shape

torch.Size([1, 3, 768])

In [299]:
_df = pd.read_csv("data/cedr/question-qq.tsv", names=["q", "qid", "text"], sep="\t")

In [300]:
_df.head()

,q,qid,text
0,question,1,What is internal benchmarking and external ben...
1,question,2,Is it OK to do push ups every day? When should...
2,question,3,What are the 3 types of brainstorming? What ar...
3,question,4,How many cigarettes does a chain smoker smoke ...
4,question,5,How do I apply for a charitable grant? What is...


In [238]:
new_texts = []
for idx, row in _df.iterrows():
    new_texts.append(preprocessingText(row.text, True))
_df["text"] = new_texts

In [301]:
new_texts = []
for idx, row in _df.iterrows():
    new_texts.append("?".join(row.text.split("?")[:1]))
_df["text"] = new_texts

In [302]:
# _df.to_csv("data/cedr/question-qq3-nostopword.tsv", header=False, index=False, sep="\t")
_df.to_csv("data/cedr/question-qq1.tsv", header=False, index=False, sep="\t")

In [175]:
to_crawl = [" ".join(i) for i in df[['action', 'entity']].drop_duplicates().values]

In [120]:
from run_scraper import MultiThreadScraper

In [131]:
to_crawl = ['serve barbecue sauce', 'bake banana bread']

In [132]:
s = MultiThreadScraper(to_crawl, "data/questions/query2/", "fail")
s.run_scraper()

100%|██████████| 2/2 [00:00<00:00, 29.56it/s]


Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='kno-ftr']//div/following-sibling::a[text()='Feedback']"}
  (Session info: headless chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='kno-ftr']//div/following-sibling::a[text()='Feedback']"}
  (Session info: headless chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='kno-ftr']//div/following-sibling::a[text()='Feedback']"}
  (Session info: headless chrome=79.0.3945.130)



In [668]:
title_to_crawl = []
for query in to_crawl:
    title = df_bm25[df_bm25.text.str.contains(query[:10])].text.values[0].replace(query+" ", "")
    title_to_crawl.append(title)
s = MultiThreadScraper(title_to_crawl, "data/questions/wiki/", "fail")
s.run_scraper()


100%|██████████| 229/229 [38:11<00:00, 10.01s/it]


In [29]:
import wikipedia
import urllib

In [56]:
notfound = []
bugs = {"Lockheed_C-5_Galaxy":"Lockheed C-5 Galaxy", "IMac":"IMac computer", "Diet_Coke": "Diet_Coke drink", "Copper": "Copper science", "Prison_Break_(season_2)": "Prison Break (season 2)", "Showtime_%28film%29":"Showtime film", "The_Wizard_of_Oz_%281939_film%29": "The Wizard of Oz film", "The_Pyramid_%28film%29":"The Pyramid film", "Prom_Night_%281980_film%29":"Prom Night film", "Law_%26_Order%3A_Special_Victims_Unit": "Law Order Special film", "Mary_Poppins_%28film%29": "Mary Poppins film", "Shock_%28circulatory%29": "Shock circulatory", "DNA": "DNA science", "Skin": "Skin science", "BP":"BP company", "Jason_Kidd": "Jason Kidd person", "Jean-Claude_Van_Damme": "Jean Claude Van Damme", "Alberta": "Alberta place", "Himalayas": "Himalayas place", "Kings_Island":"KingsIsland", "Mount_Everest": "MountEverest", "Predator_%28film%29": "Predator_%28film%29", "Braveheart": "Braveheart film", "Brainstorming": "Brain storming", "Give_Me_Five!": "Give_Me_Five! music"}
for idx, row in df[['query_id', 'url', 'entityType']].drop_duplicates().iterrows():
    try:
        if row['url'] in bugs:
            with open("data/cedr/wikipedia.tsv", 'a') as fd:
                fd.write("query\t"+str(row['query_id'])+"\t"+preprocessingText(wikipedia.summary(urllib.parse.unquote(bugs[row['url']])))+"\n")
        else:
            with open("data/cedr/wikipedia.tsv", 'a') as fd:
                fd.write("query\t"+str(row['query_id'])+"\t"+preprocessingText(wikipedia.summary(urllib.parse.unquote(row['url'])))+"\n")
    except:
        print(row["url"])
        notfound.append((row['query_id'], row['url'], row['entityType']))
        continue

In [71]:
for i in notfound:
    if i[1] not in bugs:
        print(i)